# Przygotowanie

In [1]:
import pandas as pd
import numpy as np
from gensim.models import Word2Vec

import tensorflow
from tensorflow import keras
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from keras.models import Sequential
from keras.layers import Activation, Dense, Dropout, Embedding, Flatten, Conv1D, MaxPooling1D, LSTM, Bidirectional, SpatialDropout1D
from keras import utils
from keras.callbacks import ReduceLROnPlateau, EarlyStopping

Using TensorFlow backend.


In [2]:
import os
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '2'

In [3]:
from const import W2V_MODEL_PATH, PREPROCESSED_TEST_PATH, PREPROCESSED_TRAIN_PATH

Wczytywanie danych

In [4]:
df_train = pd.read_pickle(PREPROCESSED_TRAIN_PATH)
df_test = pd.read_pickle(PREPROCESSED_TEST_PATH)

Wczytywanie w2v

In [5]:
w2v_model = Word2Vec.load(W2V_MODEL_PATH)

# Odzielanie danych od labeli

In [6]:
Y_train = df_train.target.to_numpy().reshape(-1,1)
Y_test = df_test.target.to_numpy().reshape(-1,1)

Jeszcze musimy zamienić lity tekst na tokeny. W tym celu skorzystamy z klasy __Tokenizera__ z `kerasa`.

In [7]:
tokenizer = Tokenizer()
tokenizer.fit_on_texts(df_train.text)

X_train = pad_sequences(tokenizer.texts_to_sequences(df_train.text), maxlen = 50)
X_test = pad_sequences(tokenizer.texts_to_sequences(df_test.text), maxlen = 50)

In [8]:
print("X_train", X_train.shape)
print("Y_train", Y_train.shape)
print()
print("X_test", X_test.shape)
print("Y_test", Y_test.shape)

X_train (1280000, 50)
Y_train (1280000, 1)

X_test (320000, 50)
Y_test (320000, 1)


In [9]:
X_train[0]

array([   0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
          0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
          0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
          0,    0,    0,    0,    0,    0,    0,    0,    0,   77,   41,
        240,   25,  256, 2426,   77,   41])

In [10]:
Y_train[0]

array([0], dtype=int64)

In [11]:
def create_embedding_layer():
    vocab_size = len(tokenizer.word_index) + 1
    embedding_matrix = np.zeros((vocab_size, 300))
    for word, i in tokenizer.word_index.items():
        if word in w2v_model.wv:
            embedding_matrix[i] = w2v_model.wv[word]
    return Embedding(vocab_size, 300, weights = [embedding_matrix], input_length = 50, trainable = False)

# Model

In [12]:
models = []

In [13]:
print("Training on GPU...") if tensorflow.config.list_physical_devices('GPU') else print("Training on CPU...")

Training on GPU...


### LSTM_V1
- własny embedding
- LSTM 100

In [14]:
model = Sequential(name="LSTM_V1")
model.add(create_embedding_layer())
model.add(Dropout(0.5))
model.add(LSTM(100, dropout = 0.2, recurrent_dropout = 0.2))
model.add(Dense(1, activation = 'sigmoid'))

models.append(model)

### LSTM_V2
- własny embedding
- convy
- dwukierunkowy LSTM 64
- największy dens

In [15]:
model = Sequential(name="LSTM_V2")
model.add(create_embedding_layer())
model.add(SpatialDropout1D(0.2))
model.add(Conv1D(64, 5, activation='relu'))
model.add(Bidirectional(LSTM(64, dropout=0.2, recurrent_dropout=0.2)))
model.add(Dense(512, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(512, activation='relu'))
model.add(Dense(1, activation = 'sigmoid'))

models.append(model)

### LSTM_V3
- embedding uczony na bierząco
- LSTM 50
- najmniejsza sieć

In [16]:
model = Sequential(name="LSTM_V3")
model.add(Embedding((len(tokenizer.word_index) + 1), 32, input_length=50))
model.add(SpatialDropout1D(0.25))
model.add(LSTM(50, dropout=0.5, recurrent_dropout=0.5))
model.add(Dropout(0.2))
model.add(Dense(1, activation = 'sigmoid'))

models.append(model)

In [17]:
for model in models:
    model.compile(loss = 'binary_crossentropy', optimizer = "adam", metrics = ['accuracy'])
    
    callbacks = [ 
        ReduceLROnPlateau(monitor = 'val_loss', patience = 5),
        EarlyStopping(monitor = 'val_acc', patience = 5)
    ]
    
    history = model.fit(X_train, Y_train,
                        batch_size = 1024,
                        epochs = 30,
                        validation_split = 0.1,
                        verbose = 1,
                        callbacks=callbacks)
    
    np.save("{}_history.npy".format(model.name), history.history)
    model.save("{}.model".format(model.name))
    

Train on 1152000 samples, validate on 128000 samples
Epoch 1/30
1152000/1152000 [==============================] - 240s 208us/step - loss: 0.5255 - accuracy: 0.7360 - val_loss: 0.4803 - val_accuracy: 0.7694
Epoch 2/30


C:\Users\LukMcCall\anaconda3\envs\sentimental140\lib\site-packages\keras\callbacks\callbacks.py:846: RuntimeWarning: Early stopping conditioned on metric `val_acc` which is not available. Available metrics are: val_loss,val_accuracy,loss,accuracy,lr
  (self.monitor, ','.join(list(logs.keys()))), RuntimeWarning


1152000/1152000 [==============================] - 239s 208us/step - loss: 0.4991 - accuracy: 0.7533 - val_loss: 0.4692 - val_accuracy: 0.7751
Epoch 3/30
1152000/1152000 [==============================] - 239s 208us/step - loss: 0.4897 - accuracy: 0.7595 - val_loss: 0.4637 - val_accuracy: 0.7775
Epoch 4/30
1152000/1152000 [==============================] - 239s 208us/step - loss: 0.4834 - accuracy: 0.7640 - val_loss: 0.4573 - val_accuracy: 0.7823
Epoch 5/30
1152000/1152000 [==============================] - 239s 207us/step - loss: 0.4789 - accuracy: 0.7670 - val_loss: 0.4551 - val_accuracy: 0.7840
Epoch 6/30
1152000/1152000 [==============================] - 239s 207us/step - loss: 0.4750 - accuracy: 0.7694 - val_loss: 0.4522 - val_accuracy: 0.7847
Epoch 7/30
1152000/1152000 [==============================] - 239s 208us/step - loss: 0.4722 - accuracy: 0.7715 - val_loss: 0.4501 - val_accuracy: 0.7866
Epoch 8/30
1152000/1152000 [==============================] - 239s 208us/step - loss: 0

Epoch 25/30
1152000/1152000 [==============================] - 401s 348us/step - loss: 0.4331 - accuracy: 0.7959 - val_loss: 0.4381 - val_accuracy: 0.7939
Epoch 26/30
1152000/1152000 [==============================] - 401s 348us/step - loss: 0.4323 - accuracy: 0.7963 - val_loss: 0.4383 - val_accuracy: 0.7938
Epoch 27/30
1152000/1152000 [==============================] - 401s 348us/step - loss: 0.4323 - accuracy: 0.7962 - val_loss: 0.4388 - val_accuracy: 0.7944
Epoch 28/30
1152000/1152000 [==============================] - 401s 348us/step - loss: 0.4321 - accuracy: 0.7963 - val_loss: 0.4388 - val_accuracy: 0.7946
Epoch 29/30
1152000/1152000 [==============================] - 402s 349us/step - loss: 0.4319 - accuracy: 0.7966 - val_loss: 0.4391 - val_accuracy: 0.7931
Epoch 30/30
1152000/1152000 [==============================] - 401s 348us/step - loss: 0.4312 - accuracy: 0.7970 - val_loss: 0.4384 - val_accuracy: 0.7938


C:\Users\LukMcCall\anaconda3\envs\sentimental140\lib\site-packages\tensorflow\python\framework\indexed_slices.py:434: UserWarning: Converting sparse IndexedSlices to a dense Tensor of unknown shape. This may consume a large amount of memory.
  "Converting sparse IndexedSlices to a dense Tensor of unknown shape. "


Train on 1152000 samples, validate on 128000 samples
Epoch 1/30
1152000/1152000 [==============================] - 217s 189us/step - loss: 0.5035 - accuracy: 0.7562 - val_loss: 0.4673 - val_accuracy: 0.7800
Epoch 2/30
1152000/1152000 [==============================] - 216s 187us/step - loss: 0.4590 - accuracy: 0.7867 - val_loss: 0.4614 - val_accuracy: 0.7826
Epoch 3/30
1152000/1152000 [==============================] - 216s 188us/step - loss: 0.4435 - accuracy: 0.7947 - val_loss: 0.4598 - val_accuracy: 0.7821
Epoch 4/30
1152000/1152000 [==============================] - 217s 188us/step - loss: 0.4322 - accuracy: 0.8007 - val_loss: 0.4594 - val_accuracy: 0.7818
Epoch 5/30
1152000/1152000 [==============================] - 216s 188us/step - loss: 0.4230 - accuracy: 0.8055 - val_loss: 0.4608 - val_accuracy: 0.7823
Epoch 6/30
1152000/1152000 [==============================] - 216s 188us/step - loss: 0.4150 - accuracy: 0.8091 - val_loss: 0.4633 - val_accuracy: 0.7818
Epoch 7/30
1152000/1152